# export tiles for gain and loss
Based on [ee code](https://code.earthengine.google.com/8423c718d647f15ab3e9c31868489fd4?noload=true)

In [255]:
import ee
import geemap
%run utils.ipynb

In [256]:
# Trigger the authentication flow.
#ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate()


Successfully saved authorization token.


In [257]:
# Initialize the library.
ee.Initialize()

### Data Assets

In [258]:
gcs_bucket = 'mangrove_atlas'
version = 'v3'
dataset = [f'mangrove_extent_gain-{version}', f'mangrove_extent_loss-{version}']

In [273]:
## Assets selection
# region for export
data_year_range = [2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
region = ee.Geometry.BBox(-180, -33, 0, 34);
# gain
gain = ee.ImageCollection(
    f"projects/global-mangrove-watch/land-cover/{dataset[0]}")
# loss
loss = ee.ImageCollection(
    f"projects/global-mangrove-watch/land-cover/{dataset[1]}")

In [260]:
# styles
gain_ramp ='''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#a6cb10" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''
loss_ramp = '''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#eb6240" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''

### Vizualize the data prior exporting it

In [274]:
Map = geemap.Map(center=(-5, 39), zoom=5, basemap='HYBRID')

Map.addLayer(ee.Image(gain.first().clip(region).sldStyle(gain_ramp)), {}, 'Gain', True, 1)
Map.addLayer(ee.Image(loss.first().sldStyle(loss_ramp)), {}, 'Loss', True, 1)
Map.addLayer(region, {}, 'mask', True, 1)

Map

Map(center=[-5, 39], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

### Export map Tiles

In [14]:
# Create a list of tasks to export the map tiles
gain_export_tasks = exportMapTasks(gain, 'gain', gcs_bucket, data_year_range, 
            region, gain_ramp, 0, 12, key = 'end_year')

In [15]:
# Create a list of tasks to export the map tiles
loss_export_tasks = exportMapTasks(loss, 'loss', gcs_bucket, data_year_range, 
            region, loss_ramp, 0, 12, key = 'end_year')

In [16]:
tasks_list = gain_export_tasks + loss_export_tasks

In [17]:
len(tasks_list)

20

In [18]:
batchExecute(tasks_list)

[<Task RUWYFFXFJICN35MPE4CPNZXS EXPORT_TILES: gain_2007 (UNSUBMITTED)>,
 <Task BRYWWUFZ25JBIOV6F6UZJMPN EXPORT_TILES: gain_2008 (UNSUBMITTED)>,
 <Task OV6C3OPRLVMBPZSLGYYFVDCH EXPORT_TILES: gain_2009 (UNSUBMITTED)>,
 <Task OCCRIZKKTPQXLZHO6EVC6XD5 EXPORT_TILES: gain_2010 (UNSUBMITTED)>,
 <Task GIEQHME6HR7KHCTAJV3VQQNB EXPORT_TILES: gain_2015 (UNSUBMITTED)>,
 <Task 3VH2HX3QKOETSATJE6JT6GT5 EXPORT_TILES: gain_2016 (UNSUBMITTED)>,
 <Task TVWYQYYJ63XL67XR54RHWVYS EXPORT_TILES: gain_2017 (UNSUBMITTED)>,
 <Task EECWIB2MLKNATCTNJLIOT2NR EXPORT_TILES: gain_2018 (UNSUBMITTED)>,
 <Task VLESTD6TLFWRUXAF7CPAEUJC EXPORT_TILES: gain_2019 (UNSUBMITTED)>,
 <Task DNA4B6GEOESZWETI4QVE7RV3 EXPORT_TILES: gain_2020 (UNSUBMITTED)>,
 <Task 4YRJCFTQVT5QYWEP2NSYI7NN EXPORT_TILES: loss_2007 (UNSUBMITTED)>,
 <Task J5AA3QXN2OWU7EXMYMR72IKX EXPORT_TILES: loss_2008 (UNSUBMITTED)>,
 <Task LDV2XIFCKFQB75QTXJBC5NTQ EXPORT_TILES: loss_2009 (UNSUBMITTED)>,
 <Task ZDDIAG5CD7JG6Q3AQ3DOF5LP EXPORT_TILES: loss_2010 (UNSUBMI

## Create vector tiles for gain and loss

In [15]:
my_asset = gain.first()

# Get nominal scale
ns = my_asset.projection().nominalScale().getInfo()
vector_reducer = {
            'reducer':ee.Reducer.countEvery(), 
                'geometry': region, 
                'scale': ns, 
                'geometryType': 'polygon', 
                'eightConnected': True, 
                'labelProperty': 'gain', 
                'bestEffort': True, 
                'maxPixels': 1e13, 
                'tileScale': 4, 
                'geometryInNativeProjection': False}

new_vector = ee.FeatureCollection(my_asset.reduceToVectors(**vector_reducer))

task = ee.batch.Export.table.toAsset(new_vector,'test_table', 'projects/global-mangrove-watch/test_vector', 1e6)
task.start()

# Export the vectorized FeatureCollection as a FeatureView asset.
task = ee.batch.Export.table.toFeatureView(**{
  'collection': new_vector,
  'assetId': 'projects/global-mangrove-watch/test_vector',
  'description': 'test_vector',
  'maxFeaturesPerTile': 1500,
  'thinningStrategy': 'HIGHER_DENSITY',
  'thinningRanking': ['REP_AREA DESC'],
  'zOrderRanking': ['REP_AREA DESC'],
})

#task.start()

In [18]:
new_vector.getInfo()

EEException: Computation timed out.

In [17]:
Map = geemap.Map(center=(-7.998, 39.4767), zoom=9, basemap='HYBRID')
vis_params = {
    'min': 1,
    'max': 1,
    'palette': ['eb6240'],
}
Map.addLayer(my_asset.sldStyle(gain_ramp), {}, 'gain', True, 1)
Map.addLayer(new_vector, {}, 'gain_vector', True, 1)
display(Map)

Map(center=[-7.998, 39.4767], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…